# MNIST DATASET PROJECT

Sean Moylan
<br>
G00299424

***

To get started I downloaded the MNIST dataset from http://yann.lecun.com/exdb/mnist/
<br>
I first unziped the files into the /data directory to avoid using a library to extract from within this Notebook

# Reading bytes from file

In [1]:
with open('data/t10k-images.idx3-ubyte', 'rb') as f:
    file_content = f.read()

FileNotFoundError: [Errno 2] No such file or directory: 'data/t10k-images.idx3-ubyte'